In [6]:
# Cell 1
from dotenv import load_dotenv
import os

load_dotenv()  # 从 .env 加载环境变量

# 检查 API Key 是否加载成功
api_key = os.getenv("DASHSCOPE_API_KEY")
if not api_key:
    raise ValueError("❌ 请确保 .env 文件中设置了 DASHSCOPE_API_KEY")
else:
    print("✅ DASHSCOPE_API_KEY 已成功加载"+api_key)

✅ DASHSCOPE_API_KEY 已成功加载sk-8e578ce765d5464b8f183e3afa2c8250


In [7]:
# embedding/qwen_embedding.py
import requests
import os

class QwenEmbedding:
    def __init__(self):
        self.api_key = os.getenv("DASHSCOPE_API_KEY")
        if not self.api_key:
            raise ValueError("❌ 请设置 DASHSCOPE_API_KEY")
        self.model = os.getenv("EMBEDDING_MODEL", "text-embedding-v1")
        self.url = "https://dashscope.aliyuncs.com/compatible-mode/v1/embeddings"

    def embed_batch(self, texts: list[str]) -> list[list[float]]:
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        payload = {
            "model": self.model,
            "input": texts
        }
        print("📤 发送 Embedding 请求:")
        print("  URL:", self.url)
        print("  Model:", self.model)
        print("  文本数量:", len(texts))
        print("-" * 50)
        response = requests.post(self.url, headers=headers, json=payload)
        response.raise_for_status()
        data = response.json()
        return [item["embedding"] for item in data["data"]]

In [8]:
# 模拟 settings 模块
class Settings:
    CHROMA_PERSIST_DIR = os.getenv("CHROMA_PERSIST_DIR", "./chroma_db")
    TOP_K = int(os.getenv("TOP_K", "2"))

settings = Settings()

In [9]:
# vector_store/chroma_store.py
import chromadb
from chromadb.config import Settings as ChromaSettings
from chromadb import EmbeddingFunction
# 注意：这里直接使用上面定义的 QwenEmbedding 和 settings

class QwenEmbeddingFunction(EmbeddingFunction):
    def __init__(self):
        self.qwen_emb = QwenEmbedding()
    def __call__(self, input: list[str]) -> list[list[float]]:
        return self.qwen_emb.embed_batch(input)

class ChromaVectorStore:
    def __init__(self, collection_name: str = "demo_rag"):
        self.collection_name = collection_name
        self.client = chromadb.PersistentClient(
            path=settings.CHROMA_PERSIST_DIR,
            settings=ChromaSettings(allow_reset=False)
        )
        self.embedding_function = QwenEmbeddingFunction()
        self.collection = self._get_or_create_collection()

    def _get_or_create_collection(self):
        try:
            return self.client.get_collection(name=self.collection_name)
        except:
            return self.client.create_collection(
                name=self.collection_name,
                embedding_function=self.embedding_function
            )

    def add_documents(self, documents: list[str], ids=None):
        if ids is None:
            ids = [f"doc_{i}" for i in range(len(documents))]
        self.collection.add(documents=documents, ids=ids)

    def similarity_search(self, query: str, k=2) -> list[str]:
        results = self.collection.query(query_texts=[query], n_results=k)
        return results["documents"][0]

In [10]:
# llm/qwen_llm.py
import requests
import os

class QwenLLM:
    def __init__(self):
        self.api_key = os.getenv("DASHSCOPE_API_KEY")
        if not self.api_key:
            raise ValueError("❌ 请设置 DASHSCOPE_API_KEY")
        self.model = os.getenv("LLM_MODEL", "qwen-max")
        self.url = "https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions"

    def generate(self, prompt: str) -> str:
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        payload = {
            "model": self.model,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.7
        }
        print("📤 发送 LLM 请求:")
        print("  URL:", self.url)
        print("  Model:", self.model)
        print("  Prompt 长度:", len(prompt))
        print("-" * 50)
        response = requests.post(self.url, headers=headers, json=payload)
        response.raise_for_status()
        data = response.json()
        return data["choices"][0]["message"]["content"].strip()

In [11]:
# 主程序逻辑
INITIAL_DOCS = [
    "中电金信软件有限公司（简称“中电金信”）是中国电子信息产业集团有限公司（CEC，中国电子）旗下专注于金融科技领域的高科技企业。",
    "Chroma 是一个开源的向量数据库，专为 AI 应用设计，支持嵌入、检索和存储。",
    "RAG（Retrieval-Augmented Generation）是一种结合信息检索与文本生成的技术，能显著提升问答系统的准确性。",
    "DashScope 是阿里云提供的模型开放平台，支持 Qwen 系列模型的 API 调用。"
]

print("🚀 初始化 RAG 系统...\n")

# 初始化向量数据库
vector_store = ChromaVectorStore(collection_name="demo_rag")
vector_store.add_documents(INITIAL_DOCS)
print(f"✅ 已将 {len(INITIAL_DOCS)} 篇文档存入 Chroma 向量库\n")

# 初始化 LLM
llm = QwenLLM()

# 用户提问
query = "中电金信 是什么？"
print(f"❓ 用户问题: {query}\n")

# 检索相关文档
retrieved_docs = vector_store.similarity_search(query, k=2)
context = "\n".join(retrieved_docs)
print("🔍 检索到的相关上下文:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"  [{i}] {doc}")
print()

# 构造 Prompt 并生成答案
prompt = f"""你是一个智能助手，请根据以下上下文回答问题。如果上下文不足以回答，请说“我不知道”。
上下文：
{context}
问题：{query}
回答："""
answer = llm.generate(prompt)

print("🤖 Qwen 回答:")
print(answer)

🚀 初始化 RAG 系统...

📤 发送 Embedding 请求:
  URL: https://dashscope.aliyuncs.com/compatible-mode/v1/embeddings
  Model: text-embedding-v1
  文本数量: 4
--------------------------------------------------
✅ 已将 4 篇文档存入 Chroma 向量库

❓ 用户问题: 中电金信 是什么？

📤 发送 Embedding 请求:
  URL: https://dashscope.aliyuncs.com/compatible-mode/v1/embeddings
  Model: text-embedding-v1
  文本数量: 1
--------------------------------------------------
🔍 检索到的相关上下文:
  [1] 中电金信软件有限公司（简称“中电金信”）是中国电子信息产业集团有限公司（CEC，中国电子）旗下专注于金融科技领域的高科技企业。
  [2] Chroma 是一个开源的向量数据库，专为 AI 应用设计，支持嵌入、检索和存储。

📤 发送 LLM 请求:
  URL: https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions
  Model: qwen-max
  Prompt 长度: 170
--------------------------------------------------
🤖 Qwen 回答:
中电金信软件有限公司（简称“中电金信”）是中国电子信息产业集团有限公司（CEC，中国电子）旗下的一家高科技企业，专注于金融科技领域。这意味着它主要提供与金融服务相关的技术解决方案和服务。
